In [1]:
from ngboost import NGBRegressor
from scipy.stats import expon

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from ngboost.scores import *
from ngboost.distns import *
from pandas import read_csv
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
import pickle
from sklearn.tree import DecisionTreeRegressor

def backup(object, filename):
    with open(filename, 'wb') as f:
        pickle.dump(object, f, pickle.HIGHEST_PROTOCOL)

def reload(persist):
    with open(persist, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        data = pickle.load(f)
    return data

# series = read_csv('ProcessedDataCenterHall.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
#series = read_csv('ProcessedBioEngineeringPV.csv', header=0, parse_dates=True)
outputs_feature_select = reload("outputs_feature_select.pickle")



In [2]:
dataset = pd.read_csv("ngboost_dataset.gz")

In [3]:
dataset.set_index("DateTime",inplace=True)

## all features considered

In [5]:

numlookahead = 4 # create numlookahead models
ngbdict = {}
for _ in range(1,numlookahead):
    X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:-1*numlookahead*2], dataset.iloc[:,-1*numlookahead*2+(_-1)*2:-1*numlookahead*2+_*2], test_size=0.2, shuffle=False)

    ngbdict["ngb_{}".format(_)] = NGBRegressor(Dist=MultivariateNormal(k=2), Score=LogScore, minibatch_frac=0.2, n_estimators=800, verbose=True, random_state=2022).fit(X_train, Y_train, X_val=X_test, Y_val=Y_test)

[iter 0] loss=7.9877 val_loss=8.0068 scale=1.0000 norm=40.8008
[iter 100] loss=6.7058 val_loss=6.6950 scale=1.0000 norm=15.1599
[iter 200] loss=5.7999 val_loss=5.8276 scale=2.0000 norm=12.0286
[iter 300] loss=4.8410 val_loss=4.9867 scale=1.0000 norm=3.0493
[iter 400] loss=4.2795 val_loss=4.5398 scale=1.0000 norm=2.7683
[iter 500] loss=3.9501 val_loss=4.3400 scale=1.0000 norm=2.7749
[iter 600] loss=3.6950 val_loss=4.2860 scale=0.5000 norm=1.3280
[iter 700] loss=3.5697 val_loss=4.3004 scale=1.0000 norm=2.6962
[iter 0] loss=7.9930 val_loss=8.0140 scale=1.0000 norm=40.7399
[iter 100] loss=6.8185 val_loss=6.8992 scale=1.0000 norm=16.2044
[iter 200] loss=6.0744 val_loss=6.2206 scale=1.0000 norm=7.4349
[iter 300] loss=5.4826 val_loss=5.6883 scale=1.0000 norm=4.8381
[iter 400] loss=4.9921 val_loss=5.3341 scale=1.0000 norm=4.0575
[iter 500] loss=4.6948 val_loss=5.1582 scale=0.5000 norm=2.0181
[iter 600] loss=4.4990 val_loss=5.1277 scale=1.0000 norm=3.9098
[iter 700] loss=4.3554 val_loss=5.1378 

ValueError: at least one array or dtype is required

In [18]:
#last one (due to indexing issues)
_ = numlookahead
X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:-1*numlookahead*2], dataset.iloc[:,-1*numlookahead*2+(_-1)*2:], test_size=0.2, shuffle=False)

ngbdict["ngb_{}".format(_)] = NGBRegressor(Dist=MultivariateNormal(k=2), Score=LogScore, minibatch_frac=0.2, n_estimators=800, verbose=True, random_state=2022).fit(X_train, Y_train, X_val=X_test, Y_val=Y_test)

#save the file
backup(ngbdict,"ngbdict.pickle")

[iter 0] loss=7.9948 val_loss=8.0152 scale=1.0000 norm=40.9797
[iter 100] loss=6.9994 val_loss=7.1560 scale=1.0000 norm=18.0572
[iter 200] loss=6.3302 val_loss=6.6105 scale=1.0000 norm=9.0330
[iter 300] loss=5.8356 val_loss=6.1741 scale=1.0000 norm=6.5607
[iter 400] loss=5.3987 val_loss=5.8831 scale=1.0000 norm=5.4142
[iter 500] loss=5.1396 val_loss=5.7743 scale=1.0000 norm=5.3494
[iter 600] loss=4.9615 val_loss=5.7811 scale=0.5000 norm=2.5935
[iter 700] loss=4.8810 val_loss=5.8239 scale=0.5000 norm=2.5912


In [20]:
outputs_feature_select[0]

['realpowerin_15min_0',
 'reactivepowerin_15min_0',
 'reactivepowerin_1Hr_22',
 'realpowerin_1Hr_21',
 'realpowerin_1D_0',
 'reactivepowerin_1Hr_21',
 'realpowerin_1Hr_11',
 'realpowerin_1Hr_13',
 'reactivepowerin_1Hr_23',
 'realpowerin_1Hr_7',
 'reactivepowerin_1D_0',
 'realpowerin_1Hr_15',
 'realpowerin_1Hr_9',
 'realpowerin_1Hr_23',
 'realpowerin_15min_4',
 'realpowerin_1Hr_19',
 'reactivepowerin_1Hr_20',
 'reactivepowerin_15min_3',
 'realpowerin_1Hr_17',
 'realpowerin_15min_12',
 'reactivepowerin_15min_6',
 'realpowerin_1Hr_2']

In [30]:
datasetreduced=pd.concat([dataset[outputs_feature_select[0]],dataset.iloc[:,-1*numlookahead*2:]],axis=1)

## feature reduced

In [31]:
ngbdict = {}
for _ in range(1,numlookahead):
    X_train, X_test, Y_train, Y_test = train_test_split(datasetreduced.iloc[:,:-1*numlookahead*2], datasetreduced.iloc[:,-1*numlookahead*2+(_-1)*2:-1*numlookahead*2+_*2], test_size=0.2, shuffle=False)

    ngbdict["ngb_{}".format(_)] = NGBRegressor(Dist=MultivariateNormal(k=2), Score=LogScore, minibatch_frac=0.2, n_estimators=800, verbose=True, random_state=2022).fit(X_train, Y_train, X_val=X_test, Y_val=Y_test)

[iter 0] loss=7.9877 val_loss=8.0109 scale=1.0000 norm=40.8008
[iter 100] loss=6.7509 val_loss=6.7899 scale=1.0000 norm=15.6061
[iter 200] loss=5.8871 val_loss=5.9890 scale=1.0000 norm=6.4282
[iter 300] loss=5.1354 val_loss=5.3346 scale=1.0000 norm=3.8271
[iter 400] loss=4.6521 val_loss=4.9595 scale=1.0000 norm=3.5031
[iter 500] loss=4.3399 val_loss=4.7883 scale=1.0000 norm=3.4173
[iter 600] loss=4.1282 val_loss=4.7388 scale=1.0000 norm=3.3781
[iter 700] loss=4.0190 val_loss=4.7368 scale=1.0000 norm=3.3605
[iter 0] loss=7.9930 val_loss=8.0099 scale=1.0000 norm=40.7399
[iter 100] loss=6.8815 val_loss=7.0347 scale=1.0000 norm=16.6698
[iter 200] loss=6.1975 val_loss=6.4066 scale=1.0000 norm=8.0120
[iter 300] loss=5.6193 val_loss=5.9201 scale=1.0000 norm=5.4241
[iter 400] loss=5.1942 val_loss=5.6204 scale=1.0000 norm=4.7859
[iter 500] loss=4.9363 val_loss=5.4851 scale=1.0000 norm=4.7708
[iter 600] loss=4.7641 val_loss=5.4542 scale=1.0000 norm=4.6982
[iter 700] loss=4.6679 val_loss=5.4509 s

In [32]:
_ = numlookahead
X_train, X_test, Y_train, Y_test = train_test_split(datasetreduced.iloc[:,:-1*numlookahead*2], datasetreduced.iloc[:,-1*numlookahead*2+(_-1)*2:], test_size=0.2, shuffle=False)

ngbdict["ngb_{}".format(_)] = NGBRegressor(Dist=MultivariateNormal(k=2), Score=LogScore, minibatch_frac=0.2, n_estimators=800, verbose=True, random_state=2022).fit(X_train, Y_train, X_val=X_test, Y_val=Y_test)

#save the file
backup(ngbdict,"ngbdictreduced.pickle")

[iter 0] loss=7.9948 val_loss=8.0252 scale=0.5000 norm=20.4899
[iter 100] loss=7.0170 val_loss=7.1741 scale=1.0000 norm=17.9699
[iter 200] loss=6.3641 val_loss=6.6380 scale=1.0000 norm=9.1635
[iter 300] loss=5.8885 val_loss=6.2201 scale=1.0000 norm=6.7913
[iter 400] loss=5.4823 val_loss=5.9553 scale=1.0000 norm=5.7286
[iter 500] loss=5.2310 val_loss=5.8595 scale=1.0000 norm=5.6279
[iter 600] loss=5.0885 val_loss=5.8493 scale=0.5000 norm=2.7707
[iter 700] loss=5.0197 val_loss=5.8565 scale=0.5000 norm=2.7832


### all features below

[iter 0] loss=7.9948 val_loss=8.0152 scale=1.0000 norm=40.9797 \
[iter 100] loss=6.9994 val_loss=7.1560 scale=1.0000 norm=18.0572 \
[iter 200] loss=6.3302 val_loss=6.6105 scale=1.0000 norm=9.0330 \
[iter 300] loss=5.8356 val_loss=6.1741 scale=1.0000 norm=6.5607 \
[iter 400] loss=5.3987 val_loss=5.8831 scale=1.0000 norm=5.4142 \
[iter 500] loss=5.1396 val_loss=5.7743 scale=1.0000 norm=5.3494 \
[iter 600] loss=4.9615 val_loss=5.7811 scale=0.5000 norm=2.5935 \
[iter 700] loss=4.8810 val_loss=5.8239 scale=0.5000 norm=2.5912

### reduced features below

[iter 0] loss=7.9948 val_loss=8.0252 scale=0.5000 norm=20.4899 \
[iter 100] loss=7.0170 val_loss=7.1741 scale=1.0000 norm=17.9699 \
[iter 200] loss=6.3641 val_loss=6.6380 scale=1.0000 norm=9.1635 \
[iter 300] loss=5.8885 val_loss=6.2201 scale=1.0000 norm=6.7913 \
[iter 400] loss=5.4823 val_loss=5.9553 scale=1.0000 norm=5.7286 \
[iter 500] loss=5.2310 val_loss=5.8595 scale=1.0000 norm=5.6279 \
[iter 600] loss=5.0885 val_loss=5.8493 scale=0.5000 norm=2.7707 \
[iter 700] loss=5.0197 val_loss=5.8565 scale=0.5000 norm=2.7832
### from the runs, by inspection, it appears that the validation loss is lower when all features are used

In [ ]:
# # Deterministic Values
# print("Y_preds: ", Y_preds)
# print("Y_dist: ", Y_dists)

# # Probabilistic Values
# mean = Y_dists.mean
# sample = Y_dists.rv()
# scipy_list = Y_dists.scipy_distribution()

# print("mean: ", mean)
# print("sample: ", sample)

# # test Mean Squared Error
# test_MSE = mean_squared_error(Y_preds, Y_test)
# print('Test MSE', test_MSE)

# # test Negative Log Likelihood
# test_NLL = -Y_dists.logpdf(Y_test).mean()
# print('Test NLL', test_NLL)


In [39]:
#mean of vectorized errors
ngbdict = reload("ngbdict.pickle")



In [53]:
from numpy import linalg as LA

In [63]:
combine_norms = []
for _ in range(1,numlookahead):
    X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:-1*numlookahead*2], dataset.iloc[:,-1*numlookahead*2+(_-1)*2:-1*numlookahead*2+_*2], test_size=0.2, shuffle=False)
    Y_preds = ngbdict['ngb_{}'.format(_)].predict(X_test)
    combine_norms.append(np.mean(np.apply_along_axis(LA.norm,1,Y_preds-Y_test.values)))

In [64]:
_ = 4
X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:-1*numlookahead*2], dataset.iloc[:,-1*numlookahead*2+(_-1)*2:], test_size=0.2, shuffle=False)
Y_preds = ngbdict['ngb_{}'.format(_)].predict(X_test)
combine_norms.append(np.mean(np.apply_along_axis(LA.norm,1,Y_preds-Y_test.values)))

In [66]:
np.mean(np.array(combine_norms))

4.91896561750159

In [414]:
combine_norms

[3.0266962339931736, 4.556193487748652, 5.801472591165884, 6.291500157098652]

### mean mean loss for point prediction 4.91896561750159 with point model losses of [3.0266962339931736, 4.556193487748652, 5.801472591165884, 6.291500157098652]

### start of Region Coverage

In [67]:
# _=1
# X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:-1*numlookahead*2], dataset.iloc[:,-1*numlookahead*2+(_-1)*2:-1*numlookahead*2+_*2], test_size=0.2, shuffle=False)

In [83]:
# test_NLL = -Y_dists.logpdf(Y_test).mean()
# print('Test NLL', test_NLL)

Test NLL 4.335625794645859


In [182]:
# Y_dists = ngbdict['ngb_{}'.format(1)].pred_dist(X_test)

In [445]:
from scipy.stats import chi2

boundaryval = chi2.ppf(0.25,Y_test.shape[1]) # quartile function

def det_pointbelong(cov,yval,mean):
    """
    :param cov: covariance matrix
    :param yval: test values
    :param mean: mean value of the model at that point
    returns: Boolean value whether 
    """
    return np.matmul(np.matmul(yval-mean,np.linalg.inv(cov)),np.transpose(yval-mean)) <= boundaryval

In [446]:
# cov= Y_dists.params['scale'][0]
# yval = Y_test.iloc[0].values
# mean = Y_dists.params['loc'][0]

combine_percent_belong = []
for _ in range(1,numlookahead): # 1-3 pa lang yan
    X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:-1*numlookahead*2], dataset.iloc[:,-1*numlookahead*2+(_-1)*2:-1*numlookahead*2+_*2], test_size=0.2, shuffle=False)
    Y_dists = ngbdict['ngb_{}'.format(_)].pred_dist(X_test)
    accum_belong = []

    for ct in range(Y_dists.params['loc'].shape[0]):
        accum_belong.append(det_pointbelong(Y_dists.params['scale'][ct],Y_test.iloc[ct,:].values,Y_dists.params['loc'][ct]))
    
    dfbools = pd.DataFrame(np.unique(np.array(accum_belong),return_counts=True))
    dfbools.columns = dfbools.iloc[0,:]
    combine_percent_belong.append(dfbools.loc[1,True]/(dfbools.loc[1,True]+dfbools.loc[1,False]))

    

In [448]:
_=4

X_train, X_test, Y_train, Y_test = train_test_split(dataset.iloc[:,:-1*numlookahead*2], dataset.iloc[:,-1*numlookahead*2+(_-1)*2:], test_size=0.2, shuffle=False)
Y_dists = ngbdict['ngb_{}'.format(_)].pred_dist(X_test)
accum_belong = []

for ct in range(Y_dists.params['loc'].shape[0]):
    accum_belong.append(det_pointbelong(Y_dists.params['scale'][ct],Y_test.iloc[ct,:].values,Y_dists.params['loc'][ct]))

dfbools = pd.DataFrame(np.unique(np.array(accum_belong),return_counts=True))
dfbools.columns = dfbools.iloc[0,:]
combine_percent_belong.append(dfbools.loc[1,True]/(dfbools.loc[1,True]+dfbools.loc[1,False]))

In [449]:
combine_percent_belong

[0.255526414387411,
 0.25814911952041963,
 0.22068190333458224,
 0.21693518171599852]

### These are the fraction of datapoints belonging in the quartile region
[0.255526414387411,
 0.25814911952041963,
 0.22068190333458224,
 0.21693518171599852]